In [1]:
import pandas as pd
import re
import struct
import matplotlib.pyplot as plt
import numpy as np
import sys
import os

from scipy.signal import find_peaks
import matplotlib.colors as mcolors

current_dir = '/home/marian/CIGAR_ANALYSIS/CIGAR/notebooks'

# Build the absolute path to ../functions
functions_path = os.path.abspath(os.path.join(current_dir, '../functions'))

# Add it to sys.path
sys.path.append(functions_path)

import parse_data 
import cigar as cig

from tqdm import tqdm


/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
colors = ['royalblue', 'crimson', 'black', 'green', 'darkorange', 'brown', 'coral', 'indigo', 'magenta', 'blue']


##  Data reading

In [22]:
data = {}

source = None # None, Kr, muons, Ba
# source = 'Kr' # None, Kr, muons, Ba


In [23]:

nfiles = -1
start = 1

In [ ]:
# 1.5 bars
pressure = 1.5
data[pressure] = {'source':source,
             'temperature':'17deg',
             'nchannels': 4,
             'nevents_per_wvf': 500,
             'samples_per_waveform': 752,
             'event_header_bytes': 28,
             'sample_binning': 8e-9,
             'integration_window_min':1.8e-6,
             'integration_window_max':3e-6
             }

if source == None:
    data[pressure]['run'] = 'Run2'
elif source == 'Kr':
    data[pressure]['run'] = 'Run3'

In [ ]:
# 2.5 bars
pressure = 2.5
data[pressure] = {'source':source,
             'temperature':'14.5deg',
             'nchannels': 5,
             'nevents_per_wvf': 500,
             'samples_per_waveform': 752,
             'event_header_bytes': 28,
             'sample_binning': 8e-9,
             'integration_window_min':1.8e-6,
             'integration_window_max':3e-6
             }

if source == None:
    data[pressure]['run'] = 'Run5'
elif source == 'Kr':
    data[pressure]['run'] = 'Run6'

In [ ]:
# 3.5 bars
pressure = 3.5
data[pressure] = {'source':source,
             'temperature':'13deg',
             'nchannels': 5,
             'nevents_per_wvf': 500,
             'samples_per_waveform': 752,
             'event_header_bytes': 3036,
             'sample_binning': 8e-9,
             'integration_window_min':1.8e-6,
             'integration_window_max':3e-6
             }

if source == None:
    data[pressure]['run'] = 'Run7'
elif source == 'Kr':
    data[pressure]['run'] = 'Run8'

In [ ]:
# 4.5 bars
pressure = 4.5
data[pressure] = {'source':source,
             'temperature':'11.5deg',
             'nchannels': 10,
             'nevents_per_wvf': 500,
             'samples_per_waveform': 752,
             'event_header_bytes': 28,
             'sample_binning': 8e-9,
             'integration_window_min':1.8e-6,
             'integration_window_max':3e-6
             }

if source == None:
    data[pressure]['run'] = 'Run15'
elif source == 'Kr':
    data[pressure]['run'] = 'Run16'

In [29]:
for pressure, this_run in data.items():

    run_dir = f"/home/marian/CIGAR_ANALYSIS/CIGAR/data/{this_run['run']}"

    parse_data.checkWfs(run_dir, 
                        1, 2, 
                        this_run['nchannels'], 
                        this_run['samples_per_waveform'], 
                        this_run['event_header_bytes'], 
                        print_headers = False
                        )

/home/marian/CIGAR_ANALYSIS/CIGAR/data/Run2
Waveforms shape: (500, 4, 752)
/home/marian/CIGAR_ANALYSIS/CIGAR/data/Run5
Waveforms shape: (500, 5, 752)
/home/marian/CIGAR_ANALYSIS/CIGAR/data/Run7
Waveforms shape: (500, 5, 752)
/home/marian/CIGAR_ANALYSIS/CIGAR/data/Run15
Waveforms shape: (500, 10, 752)


In [ ]:

# Initialize an empty list to store DataFrames
dataframes = []

# Loop through each folder and read all CSV files
for i, file in enumerate(tqdm(os.listdir(run_dir)[start:nfiles],desc="Reading .bin files", unit="file")):
    if file.endswith(".bin"):
        file_path = os.path.join(run_dir, file)
        # print(file_path)
        df = parse_data.parse_wf_from_binary(file_path, channels = nchannels, 
                                             n_events = nevents_per_wvf,
                                             file_idx = i,
                                             event_header_bytes = event_header_bytes
                                             )  
        dataframes.append(df)

Reading .bin files: 100%|██████████| 6/6 [00:03<00:00,  1.58file/s]


##  Data selection

In [ ]:
params = {'is_amplified':True,
          'amp_factors':{'CH1':269,
                         'CH2':267,
                         'CH3':258,
                         'CH4':275
                         },
            'pes':True
          }